In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
# import constant data
# set concentration
concentration = np.array(
    [0, 0.025, 0.0429, 0.0875, 0.1222, 0.15, 0.2, 0.2333, 0.275]
)  # mol/L

# set other constants
T = 30 + 273.15  # K
R = 8.314  # J/(mol*K)

gamma_water = 71.18  # mN/m

In [ ]:
# data_gen
raw_data = []

for i in range(0, 9):
    raw_data.append(
        np.loadtxt(
            f'data/surface_adsorption/0{i + 1}.txt', skiprows=1, delimiter=','
        ).T[1]
    )

# plot the snapshot of the data[2]
plt.plot(raw_data[2])
plt.title('Snapshot of the data[2] (0.0429 mol/L)')
plt.xlabel('time / ms')
plt.ylabel('pressure / Pa')
plt.show()


data = []

# filter data
for _raw_data in raw_data:
    _temp = []
    _min = np.min(_raw_data)
    for i in range(0, len(_raw_data)):
        if (
            _raw_data[i] - _min < 8
            and i != 0
            and i != len(_raw_data) - 1
            and _raw_data[i] <= _raw_data[i - 1]
            and _raw_data[i] < _raw_data[i + 1]
        ):
            _temp.append(_raw_data[i])
    data.append(_temp)

# plot data
print('data length:', end=' ')
for i, d in enumerate(data):
    # inspect data length
    print(len(d), end=' ')
    idx = np.arange(0, len(d))
    model = LinearRegression()
    model.fit(idx.reshape(-1, 1), d)
    plt.plot(idx, model.predict(idx.reshape(-1, 1)), '-.', color='black', alpha=0.5)
    plt.scatter(idx, d, s=20, label=f'{concentration[i]}')
plt.legend()
plt.title('Minimal pressure of each concentration')
plt.xlabel('index')
plt.ylabel('pressure / Pa')
plt.show()

# use average as target value
avg = np.array([np.mean(d) for d in data])  # Pa

print()
print('avg:', end=' ')
for a in avg:
    print(f'{a:.3f}', end=' ')

with open('data/surface_adsorption/avg_minimal_pressure.csv', 'w') as f:
    f.write('concentration,minimal_pressure\n')  # header
    for i in range(0, len(concentration)):
        f.write(f'{concentration[i]:.4f},{avg[i]:.3f}\n')
    print()
    print('data saved to data/surface_adsorption/avg_minimal_pressure.csv!')

# clac the machine constant by zero concentration
K = gamma_water / avg[0]  # mN/(m*Pa)
print(f'K: {K:.3f}')

gamma = K * avg

gamma_ = gamma[1:]

In [ ]:
# gamma - c MLR
plt.plot(concentration[1:], gamma_, 'o')

MLR_R2s = []

print('======================')
for i in range(1, 9):
    print(f'degree: {i}')
    poly = PolynomialFeatures(degree=i)
    c_poly = poly.fit_transform(np.array(concentration[1:]).reshape(-1, 1))

    model = LinearRegression()
    model.fit(c_poly, gamma_)

    r2 = model.score(c_poly, gamma_)
    print(f'r2: {r2:.4f}')
    MLR_R2s.append(r2)

    _c_new_ = np.linspace(concentration[1], concentration[-1], 100)
    _gamma_new_ = model.predict(poly.fit_transform(np.array(_c_new_).reshape(-1, 1)))

    plt.plot(_c_new_, _gamma_new_, '--' if i <= 4 else '-.')
    print('======================')
    pass

plt.legend(['data'] + [f'degree: {i}' for i in range(1, 9)])
plt.xlabel('c')
plt.ylabel('gamma')
plt.title('gamma - c')
plt.show()

with open('data/surface_adsorption/MLR_R2.csv', 'w') as f:
    f.write('degree,R2\n')  # header
    for i in range(0, len(MLR_R2s)):
        f.write(f'{i + 1},{MLR_R2s[i]:.3f}\n')
    print('data saved to data/surface_adsorption/MLR_R2.csv!')

In [ ]:
# gamma - c, use degree=3
poly = PolynomialFeatures(degree=3)
c_poly = poly.fit_transform(np.array(concentration[1:]).reshape(-1, 1))

model = LinearRegression()
model.fit(c_poly, gamma_)

_c_new_ = np.linspace(concentration[1], concentration[-1], 100)
_gamma_new_ = model.predict(poly.fit_transform(np.array(_c_new_).reshape(-1, 1)))
print(f'r2: {model.score(c_poly, gamma_)}')

plt.plot(concentration[1:], gamma_, 'o')
plt.plot(_c_new_, _gamma_new_, '--')
plt.xlabel('c')
plt.ylabel('gamma')
plt.title('gamma - c')
plt.legend(['data', 'degree: 3'])
plt.show()


# get function
def gamma_with_c(c):
    # model.predict(poly.fit_transform(np.array(c).reshape(-1, 1)))
    coef = model.coef_
    intercept = model.intercept_
    return coef[3] * c**3 + coef[2] * c**2 + coef[1] * c + intercept


# get dgamma_dc
def dgamma_dc(c):
    coef = model.coef_
    return 3 * coef[3] * c**2 + 2 * coef[2] * c + coef[1]


Z = np.array([-c * dgamma_dc(c) for c in concentration[1:]])
print('Z:', end=' ')
for z in Z:
    print(f'{z:.3f}', end=' ')

Gamma = Z / (R * T)
print()
print('Gamma:', end=' ')
for g in Gamma:
    print(f'{g:.3f}', end=' ')

c_Gamma = concentration[1:] / Gamma
print()
print('c_Gamma:', end=' ')
for cg in c_Gamma:
    print(f'{cg:.3f}', end=' ')
print()

model_f = LinearRegression()
model_f.fit(concentration[1:].reshape(-1, 1), c_Gamma)

_c_new__ = np.linspace(concentration[1], concentration[-1], 100)
_c_Gamma_new__ = model_f.predict(np.array(_c_new__).reshape(-1, 1))

print(f'r2: {model_f.score(concentration[1:].reshape(-1, 1), c_Gamma)}')

# print the result function
coef = model.coef_
intercept = model.intercept_
print(
    f'gamma = {intercept:.3f} + {coef[1]:.3f} * c + {coef[2]:.3f} * c^2 + {coef[3]:.3f} * c^3'
)

# plot c_Gamma - c
plt.plot(concentration[1:], c_Gamma, 'o')
plt.plot(_c_new__, _c_Gamma_new__, '--')
plt.xlabel('c')
plt.ylabel('c_Gamma')
plt.title('c_Gamma - c')
plt.legend(['data', 'fit'])
plt.show()

# drop the last two data
model_f = LinearRegression()
model_f.fit(concentration[1:-2].reshape(-1, 1), c_Gamma[:-2])

print(f'r2: {model_f.score(concentration[1:-2].reshape(-1, 1), c_Gamma[:-2])}')

slope_ = model_f.coef_[0]
Gamma_inf_ = 1 / slope_ * 1e-3
print(f'Gamma_inf: {Gamma_inf_:e}')

# plot c_Gamma - c
plt.plot(concentration[1:], c_Gamma, 'o')
plt.plot(_c_new__, model_f.predict(np.array(_c_new__).reshape(-1, 1)), '--')
plt.xlabel('c')
plt.ylabel('c_Gamma')
plt.title('c_Gamma - c')
plt.legend(['data', 'fit'])
plt.show()

# print the result function
coef_f = model_f.coef_
intercept_f = model_f.intercept_
print(f'c/Gamma = {intercept_f:.3f} + {coef_f[0]:.3f} * c')

In [ ]:
# gamma - ln(c)

# clac ln(concentration)
# ignore the first one (zero concentration)
ln_concentration = np.log(concentration[1:])

# linear regression
slope, intercept, r_value, p_value, std_err = st.linregress(ln_concentration, gamma_)

print(f'slope: {slope}')
print(f'intercept: {intercept}')
print(f'r2: {r_value**2}')

Gamma_inf = -slope / R / T * 1e-3
print(f'Gamma_inf: {Gamma_inf:e}')

_ln_c_new = np.linspace(ln_concentration[0], ln_concentration[-1], 100)
_gamma_new = [slope * x + intercept for x in _ln_c_new]

# plot gamma-ln(c)
plt.plot(ln_concentration, gamma_, 'o')
plt.plot(_ln_c_new, _gamma_new, '--')
plt.xlabel('ln(c)')
plt.ylabel('gamma')
plt.title('gamma-ln(c)')
plt.show()

# print the result function
print(f'gamma = {intercept:.3f} + {slope:.3f} * ln(c)')

# without the last two data points
_slope, _intercept, _r_value, _p_value, _std_err = st.linregress(
    ln_concentration[:-2], gamma_[:-2]
)

print(f'slope: {_slope}')
print(f'intercept: {_intercept}')
print(f'r2: {_r_value**2}')

_Gamma_inf = -_slope / R / T * 1e-3
print(f'Gamma_inf: {_Gamma_inf:e}')

__ln_c_new = np.linspace(ln_concentration[0], ln_concentration[-1], 100)
_gamma_new = [_slope * x + _intercept for x in __ln_c_new]

# plot gamma-ln(c)
plt.plot(ln_concentration, gamma_, 'o')
plt.plot(__ln_c_new, _gamma_new, '--')
plt.xlabel('ln(c)')
plt.ylabel('gamma')
plt.title('gamma-ln(c)')
plt.show()

# print the result function
print(f'gamma = {_intercept:.3f} + {_slope:.3f} * ln(c)')

In [ ]:
# write data to file
with open('data/surface_adsorption/data.csv', 'w') as f:
    f.write('concentration,P,Z,ln_c,gamma,Gamma,c_Gamma\n')  # header
    for i in range(0, len(concentration[1:])):
        f.write(
            f'{concentration[i + 1]:.4f},{avg[i + 1]:.3f},{Z[i]:.3f},{ln_concentration[i]:.4f},{gamma_[i]:.3f},{Gamma[i]:.3f},{c_Gamma[i]:.3f}\n'
        )
    print('data saved to data/surface_adsorption/data.csv!')

In [ ]:
# clac the error
N_A = 6.022e23
# use Gamma_inf_standard_max as standard reference
q_standard_min = 2.4e-19
q_standard_max = 2.9e-19

print(f'Gamma_inf: {Gamma_inf_:e}')
q_1 = 1 / Gamma_inf_ / N_A
print(f'q_1: {q_1:e}')
error_1 = (q_1 - q_standard_max) / q_standard_max * 100
print(f'error_1: {error_1:.3f}%')

print('======================')

print(f'Gamma_inf: {Gamma_inf:e}')
q_2 = 1 / Gamma_inf / N_A
print(f'q_2: {q_2:e}')
error_2 = (q_2 - q_standard_max) / q_standard_max * 100
print(f'error_2: {error_2:.3f}%')

print('======================')

print(f'Gamma_inf: {_Gamma_inf:e}')
_q_2 = 1 / _Gamma_inf / N_A
print(f'q_2: {_q_2:e}')
_error_2 = (_q_2 - q_standard_max) / q_standard_max * 100
print(f'error_2: {_error_2:.3f}%')

In [ ]:
# 2mm
rho = 1e3  # kg/m^3
g = 9.8  # m/s^2
delta_P = rho * g * 2e-3
print(f'delta_P: {delta_P:.3f} Pa')